In [11]:
import pandas as pd
import sqlite3
import os
from pathlib import Path

In [12]:
# Create loaded directory
os.makedirs('loaded', exist_ok=True)
print("Created 'loaded' directory")

Created 'loaded' directory


In [13]:
# Load transformed datasets
print("Loading transformed datasets...")
df_full = pd.read_csv(r'c:\Users\cselue\Desktop\ET_Exam_Chiadika_280\transformed\transformed_full.csv')
df_incremental = pd.read_csv(r'c:\Users\cselue\Desktop\ET_Exam_Chiadika_280\transformed\transformed_incremental.csv')

print(f"Full dataset shape: {df_full.shape}")
print(f"Incremental dataset shape: {df_incremental.shape}")

Loading transformed datasets...
Full dataset shape: (6, 13)
Incremental dataset shape: (3, 13)


In [14]:
#Load into SQLite Database
print("Loading into SQLite Database")

# Create SQLite connection
conn = sqlite3.connect('loaded/retail_data.db')
print("Connected to SQLite database")

# Load full data into SQLite
df_full.to_sql('full_retail_data', conn, if_exists='replace', index=False)
print("Full data loaded into SQLite table 'full_retail_data'")

# Load incremental data into SQLite
df_incremental.to_sql('incremental_retail_data', conn, if_exists='replace', index=False)
print("Incremental data loaded into SQLite table 'incremental_retail_data'")

Loading into SQLite Database
Connected to SQLite database
Full data loaded into SQLite table 'full_retail_data'
Incremental data loaded into SQLite table 'incremental_retail_data'


In [15]:
# Verify SQLite Load
print("Verifying SQLite Load")

# Query full data
sqlite_full_sample = pd.read_sql('SELECT * FROM full_retail_data LIMIT 5', conn)
print("Full Data Sample from SQLite:")
display(sqlite_full_sample)

# Query incremental data
sqlite_inc_sample = pd.read_sql('SELECT * FROM incremental_retail_data LIMIT 5', conn)
print("Incremental Data Sample from SQLite:")
display(sqlite_inc_sample)

# Check record counts
full_count = pd.read_sql('SELECT COUNT(*) as count FROM full_retail_data', conn).iloc[0,0]
inc_count = pd.read_sql('SELECT COUNT(*) as count FROM incremental_retail_data', conn).iloc[0,0]

print(f"Record counts in SQLite:")
print(f"Full data: {full_count} records")
print(f"Incremental data: {inc_count} records")

Verifying SQLite Load
Full Data Sample from SQLite:


,InvoiceNo,InvoiceDate,Quantity,UnitPrice,CustomerID,Description,Country,TotalAmount,Year,Month,DayOfWeek,Hour,SalesBracket
0,1001,2011-10-31,1,10.0,12345,A,United Kingdom,10.0,2011,10,Monday,0,Small (≤£10)
1,1002,2011-11-05,2,20.0,23456,B,United Kingdom,40.0,2011,11,Saturday,0,Medium (£11-£50)
2,1003,2011-11-20,1,30.0,34567,C,United Kingdom,30.0,2011,11,Sunday,0,Medium (£11-£50)
3,1001,2011-10-31,1,10.0,12345,Widget A,United Kingdom,10.0,2011,10,Monday,0,Small (≤£10)
4,1002,2011-11-05,2,20.0,23456,Widget B,United Kingdom,40.0,2011,11,Saturday,0,Medium (£11-£50)


Incremental Data Sample from SQLite:


,InvoiceNo,InvoiceDate,Quantity,UnitPrice,CustomerID,Description,Country,TotalAmount,Year,Month,DayOfWeek,Hour,SalesBracket
0,1001,2011-10-31,1,10.0,12345,Widget A,United Kingdom,10.0,2011,10,Monday,0,Small (≤£10)
1,1002,2011-11-05,2,20.0,23456,Widget B,United Kingdom,40.0,2011,11,Saturday,0,Medium (£11-£50)
2,1003,2011-11-20,1,30.0,34567,Widget C,United Kingdom,30.0,2011,11,Sunday,0,Medium (£11-£50)


Record counts in SQLite:
Full data: 6 records
Incremental data: 3 records


In [16]:
# Check schema information
print("Database Schema Information")
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
print("Tables in database:")
print(tables)

# Get column information for full_retail_data
columns_info = pd.read_sql("PRAGMA table_info(full_retail_data)", conn)
print("\nColumns in full_retail_data table:")
print(columns_info[['name', 'type']])

Database Schema Information
Tables in database:
                      name
0         full_retail_data
1  incremental_retail_data

Columns in full_retail_data table:
            name     type
0      InvoiceNo  INTEGER
1    InvoiceDate     TEXT
2       Quantity  INTEGER
3      UnitPrice     REAL
4     CustomerID  INTEGER
5    Description     TEXT
6        Country     TEXT
7    TotalAmount     REAL
8           Year  INTEGER
9          Month  INTEGER
10     DayOfWeek     TEXT
11          Hour  INTEGER
12  SalesBracket     TEXT


In [17]:
# Verify Record Counts
print("Record Count Verification")

# Check record counts for both tables
full_count = pd.read_sql('SELECT COUNT(*) as record_count FROM full_retail_data', conn).iloc[0,0]
inc_count = pd.read_sql('SELECT COUNT(*) as record_count FROM incremental_retail_data', conn).iloc[0,0]

print(f"Record counts in SQLite:")
print(f"Full data: {full_count} records")
print(f"Incremental data: {inc_count} records")

# Compare with original DataFrame sizes
print(f"\nOriginal DataFrame sizes:")
print(f"Full data: {len(df_full)} records")
print(f"Incremental data: {len(df_incremental)} records")

# Verify data integrity
if len(df_full) == full_count and len(df_incremental) == inc_count:
    print("Data integrity verified - all records loaded successfully")
else:
    print("Data count mismatch detected")

Record Count Verification
Record counts in SQLite:
Full data: 6 records
Incremental data: 3 records

Original DataFrame sizes:
Full data: 6 records
Incremental data: 3 records
Data integrity verified - all records loaded successfully


In [18]:
# Advanced Queries for Data Validation
print("Data Validation Queries")

# Check for any NULL values in key columns
print("Checking for NULL values in key columns:")
null_check_query = """
SELECT 
    SUM(CASE WHEN InvoiceNo IS NULL THEN 1 ELSE 0 END) as null_invoice,
    SUM(CASE WHEN Quantity IS NULL THEN 1 ELSE 0 END) as null_quantity,
    SUM(CASE WHEN UnitPrice IS NULL THEN 1 ELSE 0 END) as null_unitprice
FROM full_retail_data
"""
null_results = pd.read_sql(null_check_query, conn)
print(null_results)

# Check data ranges and statistics
print("\nData Statistics from SQLite:")
stats_query = """
SELECT 
    MIN(Quantity) as min_quantity,
    MAX(Quantity) as max_quantity,
    AVG(Quantity) as avg_quantity,
    MIN(UnitPrice) as min_unitprice,
    MAX(UnitPrice) as max_unitprice,
    AVG(UnitPrice) as avg_unitprice,
    MIN(TotalAmount) as min_total,
    MAX(TotalAmount) as max_total,
    AVG(TotalAmount) as avg_total
FROM full_retail_data
"""
stats_results = pd.read_sql(stats_query, conn)
print(stats_results)

Data Validation Queries
Checking for NULL values in key columns:
   null_invoice  null_quantity  null_unitprice
0             0              0               0

Data Statistics from SQLite:
   min_quantity  max_quantity  avg_quantity  min_unitprice  max_unitprice  \
0             1             2      1.333333           10.0           30.0   

   avg_unitprice  min_total  max_total  avg_total  
0           20.0       10.0       40.0  26.666667  


In [19]:
# Sample Business Queries
print("Sample Business Queries")

# Top selling products
print("Top 5 products by total sales:")
top_products_query = """
SELECT 
    Description,
    SUM(Quantity) as total_quantity,
    SUM(TotalAmount) as total_revenue,
    COUNT(*) as transaction_count
FROM full_retail_data 
GROUP BY Description 
ORDER BY total_revenue DESC 
LIMIT 5
"""
top_products = pd.read_sql(top_products_query, conn)
print(top_products)

# Sales by country
print("\nSales by country:")
country_sales_query = """
SELECT 
    Country,
    SUM(TotalAmount) as total_revenue,
    COUNT(*) as transaction_count
FROM full_retail_data 
GROUP BY Country 
ORDER BY total_revenue DESC
"""
country_sales = pd.read_sql(country_sales_query, conn)
print(country_sales)

Sample Business Queries
Top 5 products by total sales:
  Description  total_quantity  total_revenue  transaction_count
0    Widget B               2           40.0                  1
1           B               2           40.0                  1
2    Widget C               1           30.0                  1
3           C               1           30.0                  1
4    Widget A               1           10.0                  1

Sales by country:
          Country  total_revenue  transaction_count
0  United Kingdom          160.0                  6


In [20]:
# Final Database Summary
print("Database Summary")

# Get database file size
db_path = 'loaded/retail_data.db'
if os.path.exists(db_path):
    db_file_size = os.path.getsize(db_path) / (1024 * 1024)  # Size in MB
    print(f"Database file size: {db_file_size:.2f} MB")
else:
    print("Database file not found")

print(f"\nFinal Record Counts:")
print(f"Full data: {full_count} records")
print(f"Incremental data: {inc_count} records")

# Close database connection
conn.close()
print("Database connection closed")

print("\nLoad Phase Completed Successfully")
print("Data successfully loaded into SQLite database")
print("All verifications passed")
print("Database saved to: loaded/retail_data.db")

Database Summary
Database file size: 0.01 MB

Final Record Counts:
Full data: 6 records
Incremental data: 3 records
Database connection closed

Load Phase Completed Successfully
Data successfully loaded into SQLite database
All verifications passed
Database saved to: loaded/retail_data.db
